卷积神经网络 - Convolutional Neural Network
====


In [1]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

torch.manual_seed(1)  # reproducible

In [3]:
GPU_FLAG = torch.cuda.is_available()

# Hyper Parameters
EPISODES = 10
BATCH_SIZE = 1000

数据迭代器

In [4]:
import torch.utils.data as Data
import torchvision

# Mnist digits dataset
train_data = torchvision.datasets.MNIST(
    root='data/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)
test_data = torchvision.datasets.MNIST(
    root='data/',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)

train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

建立网络

## 参数初始化
### 方式1
xavier_uniform - 只能初始化weights，不能bias（维度不够，至少2维）<br>
``
if len(para.size()) >= 2:
    torch.nn.init.xavier_uniform(para)
else:
    pass
``

### 方式2
constant - 所有参数全部为0<br>
``
torch.nn.init.constant(para, val=0)
``

### 方式3
normal - 均值为0，方差为1的正态分布<br>
``torch.nn.init.normal(para)``

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=16,
                               kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=16,
                               out_channels=32,
                               kernel_size=5)
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)

        # initialize
        self.init_parameters()

        # loss & optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def init_parameters(self):
        # initialize
        for para in self.parameters():
            # xavier_uniform - 只能初始化weights，不能bias（维度不够，至少2维）
            if len(para.size()) >= 2:
                torch.nn.init.xavier_uniform(para)
            else:
                pass

    def forward(self, x):
        out = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        out = F.max_pool2d(F.relu(self.conv2(out)), (2, 2))
        out = out.view(out.size(0), -1)  # reshape
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.softmax(self.fc3(out))
        return out

    def train_model(self, x, y):
        self.train()
        self.optimizer.zero_grad()
        y_pred = self.forward(x)
        loss = self.criterion(y_pred, y)
        loss.backward()
        self.optimizer.step()
        return loss

    def get_accuracy(self):
        self.eval()
        self.accuracy = 0.0
        for _ in xrange(len(test_loader)):
            batch_images, batch_labels = test_loader.__iter__().next()
            if GPU_FLAG:
                batch_images = Variable(batch_images).cuda()
                batch_labels = Variable(batch_labels).cuda()
            else:
                batch_images = Variable(batch_images)
                batch_labels = Variable(batch_labels)
            pred = self.forward(batch_images)
            self.accuracy += (torch.max(pred, 1)[-1] == batch_labels).float().mean()
        return float(self.accuracy / len(test_loader))


convNet = ConvNet()
if GPU_FLAG:
    convNet.cuda()
else:
    pass
print(convNet)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


ConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)


训练

In [6]:
for i in xrange(1, 1 + EPISODES):
    cost = 0.0
    for _ in xrange(len(train_loader)):
        images, labels = train_loader.__iter__().next()
        if GPU_FLAG:
            x_data = Variable(images).cuda()
            y_data = Variable(labels).cuda()
        else:
            x_data = Variable(images)
            y_data = Variable(labels)
        loss = convNet.train_model(x=x_data, y=y_data)
        cost += loss
        cost /= len(train_loader)
    print('Epoch %s / %s, training cost: %s, accuracy: %s' % (i, EPISODES, float(cost), convNet.get_accuracy()))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 / 10, training cost: 0.02870197780430317, accuracy: 0.7386000156402588
Epoch 2 / 10, training cost: 0.027701640501618385, accuracy: 0.8445000648498535
Epoch 3 / 10, training cost: 0.026608150452375412, accuracy: 0.883899986743927
Epoch 4 / 10, training cost: 0.02685076929628849, accuracy: 0.8939999938011169
Epoch 5 / 10, training cost: 0.026745686307549477, accuracy: 0.8923999667167664
Epoch 6 / 10, training cost: 0.026469416916370392, accuracy: 0.8905999064445496
Epoch 7 / 10, training cost: 0.02655244991183281, accuracy: 0.8948999643325806
Epoch 8 / 10, training cost: 0.02636571414768696, accuracy: 0.8989999890327454
Epoch 9 / 10, training cost: 0.026466432958841324, accuracy: 0.901199996471405
Epoch 10 / 10, training cost: 0.026307562366127968, accuracy: 0.9014999270439148


In [7]:
correct = 0
for images, labels in test_loader:
    if GPU_FLAG:
        images = Variable(images.view(-1, 1, 28, 28)).cuda()
        labels = labels.cuda()
    else:
        pass
    outputs = convNet(images)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).float().mean()
print('Final test accuracy: %s', float(correct / len(test_loader)))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Final test accuracy: %s 0.8991999626159668
